# Week02 - Advanced Reasoning Techniques

본 노트북은 고급 추론 프롬프트 엔지니어링 기법들을 실습합니다.

## 다룰 기법들
1. **Zero-shot**: 예시 없이 직접 추론
2. **Few-shot**: 소수 예시로 패턴 학습
3. **Chain of Thought (CoT)**: 단계별 추론
4. **Least-to-Most**: 문제 분해 후 순차 해결
5. **Tree of Thoughts (ToT)**: 다중 경로 탐색
6. **ReAct**: 추론과 행동의 반복
7. **Program-Aided Language Model (PAL)**: 코드를 통한 정확한 계산

In [1]:
# 필요한 라이브러리 설치 및 import
import subprocess
import json
import re
import time
import random
import os
import math
import functools
from dotenv import load_dotenv
from typing import Dict, List, Optional, Any

# .env 파일 로드
load_dotenv()

# OpenAI 라이브러리
try:
    from openai import OpenAI
except ImportError:
    !pip install openai
    from openai import OpenAI

# 설정
client = OpenAI()  # 환경변수에서 API 키 자동 로드

# 헬퍼 함수들
def run_ollama(model: str, prompt: str) -> str:
    """Ollama 모델 실행"""
    try:
        result = subprocess.run(
            ["ollama", "run", model],
            input=prompt,
            text=True,
            capture_output=True,
            timeout=60
        )
        return result.stdout.strip()
    except subprocess.TimeoutExpired:
        return "Error: Timeout"
    except Exception as e:
        return f"Error: {str(e)}"

def run_openai(prompt: str, model: str = "gpt-5-mini", **kwargs) -> str:
    """OpenAI 모델 실행"""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            **kwargs
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

def run_openai_with_messages(messages: List[Dict], model: str = "gpt-5-mini", **kwargs) -> str:
    """OpenAI 모델 실행 (메시지 리스트 사용)"""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            **kwargs
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

## 1. Zero-shot

**핵심 아이디어**: 예시 없이 문제·출력 포맷만으로 답을 얻음

- 장점: 준비 비용↓, 일반화 테스트에 유리
- 단점: 포맷 일탈·과잉창작 위험
- 설계 팁: "**역할** + **태스크** + **출력 스키마/제약**"

In [2]:
# 1. Zero-shot - 고객 리뷰 감성 분류
zero_shot_prompt = """Role: sentiment classifier.
Task: Classify review as positive|negative|neutral and give a one-line reason.
Return ONLY JSON {label, reason}.
Review: --- 배송 빠르고 포장 깔끔, 재구매 의사 있어요 ---"""

print("=== Zero-shot 감성 분류 ===")
print("Ollama (llama3.1:8b):")
zero_shot_ollama = run_ollama("llama3.1:8b", zero_shot_prompt)
print(zero_shot_ollama)

print("\nOpenAI (gpt-5-mini):")
zero_shot_openai = run_openai(zero_shot_prompt, max_completion_tokens=120)
print(zero_shot_openai)

=== Zero-shot 감성 분류 ===
Ollama (llama3.1:8b):
```
{
  "label": "positive",
  "reason": "Expresses satisfaction with fast delivery and good packaging"
}
```

OpenAI (gpt-5-mini):



In [3]:
# Zero-shot 다른 예시들
zero_shot_examples = [
    "배송이 늦고 포장도 찢어졌어요",
    "가격은 비싸지만 품질이 좋네요",
    "보통 수준입니다. 특별하지 않아요"
]

print("=== Zero-shot 다양한 예시 테스트 ===")
for i, review in enumerate(zero_shot_examples, 1):
    prompt = f"""Role: sentiment classifier.
Task: Classify review as positive|negative|neutral and give a one-line reason.
Return ONLY JSON {{label, reason}}.
Review: --- {review} ---"""
    
    result = run_openai(prompt, max_completion_tokens=100)
    print(f"예시 {i}: {review}")
    print(f"결과: {result}\n")

=== Zero-shot 다양한 예시 테스트 ===
예시 1: 배송이 늦고 포장도 찢어졌어요
결과: 

예시 2: 가격은 비싸지만 품질이 좋네요
결과: 

예시 3: 보통 수준입니다. 특별하지 않아요
결과: 



## 2. Few-shot

**핵심 아이디어**: 소수 예시로 의도·포맷을 학습 유도

- 장점: 포맷 안정화·경계 케이스 교정
- 단점: 데이터 누수/편향·컨텍스트 사용량↑
- 설계 팁: "좋은/나쁜" 예시를 대조로 배치

In [4]:
# 2. Few-shot - 주문 의도 분류
few_shot_prompt = """Task: intent classification -> refund|exchange|question. JSON {intent}.
Examples:
- '개봉 전인데 환불 원해요' -> {"intent": "refund"}
- '사이즈 안 맞아 교환할래요' -> {"intent": "exchange"}
- '배송 언제 오나요?' -> {"intent": "question"}
Now classify: --- 색상이 사진과 달라요, 바꿀 수 있나요? ---"""

print("=== Few-shot 의도 분류 ===")
print("Ollama:")
few_shot_ollama = run_ollama("llama3.1:8b", few_shot_prompt)
print(few_shot_ollama)

print("\nOpenAI:")
few_shot_openai = run_openai(few_shot_prompt, max_completion_tokens=80)
print(few_shot_openai)

=== Few-shot 의도 분류 ===
Ollama:
A nice example!

Based on the context of the sentence, I would classify it as:

{"intent": "exchange"}

The reason is that the customer is complaining about a discrepancy between the color in the picture and the actual product, which implies they want to exchange the item for one with the correct color. The phrase "바꿀 수 있나요?" (Can I change/replace it?) further supports this interpretation.

Am I correct?

OpenAI:



In [5]:
# Few-shot 성능 비교: Zero-shot vs Few-shot
test_inputs = [
    "불량품이라서 돈 돌려받고 싶어요",
    "다른 색깔로 바꿔주세요",
    "언제까지 사용할 수 있나요?"
]

print("=== Zero-shot vs Few-shot 성능 비교 ===")

for text in test_inputs:
    print(f"\n입력: {text}")
    
    # Zero-shot
    zero_prompt = f"Classify customer intent -> refund|exchange|question. JSON only.\nText: {text}"
    zero_result = run_openai(zero_prompt, max_completion_tokens=50)
    print(f"Zero-shot: {zero_result}")
    
    # Few-shot  
    few_prompt = f"""Examples:
- '환불하고 싶어요' -> {{"intent": "refund"}}
- '교환 가능한가요?' -> {{"intent": "exchange"}}
- '배송 정보 알려주세요' -> {{"intent": "question"}}
Classify: {text}"""
    few_result = run_openai(few_prompt, max_completion_tokens=50)
    print(f"Few-shot: {few_result}")

=== Zero-shot vs Few-shot 성능 비교 ===

입력: 불량품이라서 돈 돌려받고 싶어요
Zero-shot: 
Few-shot: 

입력: 다른 색깔로 바꿔주세요
Zero-shot: 
Few-shot: 

입력: 언제까지 사용할 수 있나요?
Zero-shot: 
Few-shot: 


## 3. Chain of Thought (CoT)

**핵심 아이디어**: 단계별 추론을 유도해 오류율↓

- 장점: 복잡 산술·논리·계획 문제에 강함
- 설계 팁: "단계별로 사고하되, 마지막에 '정답만' 별도 표기"

In [6]:
# 3. Chain of Thought - 수학 문제
cot_math_prompt = """Solve step by step. Provide JSON:
{"steps": [...], "final_answer": number}
Q: 37 * 46"""

print("=== CoT 수학 문제 ===")
print("Ollama:")
cot_math_ollama = run_ollama("llama3.1:8b", cot_math_prompt)
print(cot_math_ollama)

print("\nOpenAI:")
cot_math_openai = run_openai(cot_math_prompt, max_completion_tokens=250)
print(cot_math_openai)

# 정답 확인
print(f"\n실제 정답: {37 * 46}")

=== CoT 수학 문제 ===
Ollama:
Here is the solution:

{
  "steps": [
    {
      "operator": "*",
      "operands": [
        {
          "value": 30,
          "digits": ["3", "0"]
        },
        {
          "value": 20,
          "digits": ["2", "0"]
        }
      ],
      "result": {
        "value": 600,
        "digits": ["6", "0", "0"]
      }
    },
    {
      "operator": "*",
      "operands": [
        {
          "value": 10,
          "digits": ["1", "0"]
        },
        {
          "value": 20,
          "digits": ["2", "0"]
        }
      ],
      "result": {
        "value": 200,
        "digits": ["2", "0", "0"]
      }
    },
    {
      "operator": "+",
      "operands": [
        {
          "value": 600,
          "digits": ["6", "0", "0"]
        },
        {
          "value": 200,
          "digits": ["2", "0", "0"]
        }
      ],
      "result": {
        "value": 800,
        "digits": ["8", "0", "0"]
      }
    }
  ],
  "final_answer": 800
}

OpenAI:

In [7]:
# CoT 논리 추론 문제
cot_logic_prompt = """Think step by step and provide your reasoning.
Problem: 모든 고양이는 동물이다. 솜이는 고양이다. 따라서 솜이는 동물이다.
이 논증의 유효성을 단계별로 분석하고 JSON으로 답하시오:
{"steps": [...], "conclusion": "valid|invalid", "reason": "..."}"""

print("=== CoT 논리 추론 ===")
cot_logic_result = run_openai(cot_logic_prompt, max_completion_tokens=300)
print(cot_logic_result)

=== CoT 논리 추론 ===



In [8]:
# CoT vs 직접 답변 비교
word_problem = "Sarah가 사과 24개를 가지고 있다. 친구 3명에게 똑같이 나누어주고, 자신은 3개를 더 먹었다. 남은 사과는 몇 개인가?"

print("=== CoT vs 직접 답변 비교 ===")

# 직접 답변
direct_prompt = f"문제: {word_problem}\n답:"
direct_result = run_openai(direct_prompt, max_completion_tokens=100)
print(f"직접 답변: {direct_result}")

# CoT 답변
cot_prompt = f"""문제를 단계별로 해결하세요.
JSON 형식: {{"steps": [...], "final_answer": number}}
문제: {word_problem}"""
cot_result = run_openai(cot_prompt, max_completion_tokens=200)
print(f"\nCoT 답변: {cot_result}")

# 수동 계산으로 정답 확인
# Sarah 24개 -> 친구 3명에게 나누어줌 (24/3 = 8개씩) -> 24-24 = 0개
# 아니면 Sarah가 친구들과 함께 4명이서 나누어 먹는다는 의미일 수도: 24/4 = 6개씩
# Sarah가 3개 더 먹음 -> 6-3 = 3개 남음
print("\n문제 해석에 따른 정답:")
print("해석1 (모든 사과를 나누어줌): 24-24+0-3 = -3 (불가능)")
print("해석2 (4명이서 나누어 먹음): 24/4=6, 6-3=3개 남음")

=== CoT vs 직접 답변 비교 ===
직접 답변: 

CoT 답변: 

문제 해석에 따른 정답:
해석1 (모든 사과를 나누어줌): 24-24+0-3 = -3 (불가능)
해석2 (4명이서 나누어 먹음): 24/4=6, 6-3=3개 남음


## 4. Least-to-Most

**핵심 아이디어**: 문제를 작은 하위문제로 분해 → 순서대로 해결

- 장점: 장문 추론, 제약 충돌 해결에 유리
- 설계 템플릿: 1) 하위목표 나열 2) 각 목표 해결 3) 최종 통합 답 산출

In [9]:
# 4. Least-to-Most - 복합 제약 문제
ltm_prompt = """Use least-to-most decomposition.
Problem: 예산 10만원, 3명, 비건 1명을 위한 점심 코스 추천 (2곳), 총액≤10만원

Step 1: 하위목표 나열
Step 2: 각 목표 해결  
Step 3: 통합 솔루션

Return markdown table: | 식당 | 인당가격 | 총액 | 비건옵션 | 특징 |"""

print("=== Least-to-Most 점심 추천 ===")
print("Ollama:")
ltm_ollama = run_ollama("llama3.1:8b", ltm_prompt)
print(ltm_ollama)

print("\n=== OpenAI ===")
ltm_openai = run_openai(ltm_prompt, max_completion_tokens=400)
print(ltm_openai)

=== Least-to-Most 점심 추천 ===
Ollama:
예산 10만원, 3명, 비건 1명을 위한 점심 코스 추천 (2곳)에는 다음과 같은 해결책이 있습니다.

| 식당 | 인당가격 | 총액 | 비건옵션 | 특징 |
| --- | --- | --- | --- | --- |
| 가 | 30,000원/인 | 90,000원 | ☕️ | 주말에만 운영 |
| 나 | 25,000원/인 | 75,000원 | 🌱 | 비건 식당 |

**해결 방법**

1. **하위목표**: 
    - 비용: 10만원 이내
    - 인원: 3명 (비건 1명)
    - 점심 코스 추천 (2곳)

2. **각 목표 해결**
   - **식당 1:** 가
     - 가격: 30,000원/인
     - 총 비용: 90,000원
     - 비건 옵션: ☕️
     - 특징: 주말에만 운영

   - **식당 2:** 나
     - 가격: 25,000원/인
     - 총 비용: 75,000원
     - 비건 옵션: 🌱
     - 특징: 비건 식당

3. **통합 솔루션**
   - 두 식당 중에서 인당 가격이 저렴하고 비건 옵션이 있는 나 식당을 선택합니다.
   - 총 비용은 75,000원으로 10만원 이내에 점심 코스를 해결할 수 있습니다.

결과적으로 나 식당 (비건 옵션)으로 비용은 90,000원에서 75,000원으로 줄어든 것을 확인할 수 있습니다.

=== OpenAI ===



In [10]:
# Least-to-Most 프로젝트 계획
ltm_project_prompt = """Use least-to-most approach to create a project plan.
Goal: "웹사이트 리뉴얼 프로젝트 (4주, 팀 5명, 예산 500만원)"

1) 주요 하위 작업들을 식별
2) 각 작업의 리소스와 시간 할당
3) 최종 통합 계획표 작성

Output format: JSON with {"subtasks": [...], "timeline": {...}, "final_plan": "..."}"""

print("=== Least-to-Most 프로젝트 계획 ===")
ltm_project_result = run_openai(ltm_project_prompt, max_completion_tokens=500)
print(ltm_project_result)

=== Least-to-Most 프로젝트 계획 ===



## 5. Tree of Thoughts (ToT)

**핵심 아이디어**: 여러 추론 경로(생각 가지) 생성 → 자체평가 후 최고안 선택

- 장점: 탐색적 문제(기획·전략·설계)에 강함
- 설계 템플릿: 가지 생성 N(보통 3) → 각 가지 장단점 평가 → 최종 선택

In [11]:
# 5. Tree of Thoughts - 학습 전략
tot_prompt = """Generate 3 distinct study plans using Tree of Thoughts.
Goal: "고등학생을 위한 4주 공부전략(수학·영어·과학 균형)"

For each plan:
1) Outline the approach
2) List pros and cons
3) Rate effectiveness (1-10)

Finally, pick BEST plan with 1-line rationale.

Return JSON: {"plans": [{"name": "...", "outline": "...", "pros": [...], "cons": [...], "rating": N}], "best_choice": {"name": "...", "reason": "..."}}"""

print("=== Tree of Thoughts 학습 전략 ===")
print("Ollama:")
tot_ollama = run_ollama("llama3.1:8b", tot_prompt)
print(tot_ollama)

print("\n=== OpenAI ===")
tot_openai = run_openai(tot_prompt, max_completion_tokens=700)
print(tot_openai)

=== Tree of Thoughts 학습 전략 ===
Ollama:
Here are three distinct study plans using the Tree of Thoughts approach to achieve a balanced 4-week study strategy for high school students in math, English, and science:

**Plan 1: Intensive Focus**

* **Approach:** Divide the 4 weeks into four main topics (2 in math, 1 in English, 1 in science). Each week, focus on one topic exclusively, allocating 70% of study time to it.
* **Pros:**
	+ Deep understanding of each topic
	+ Improved retention and recall
	+ Time management skills development
* **Cons:**
	+ Risk of neglecting other important topics
	+ Limited exposure to different subjects
	+ Potential burnout due to intense focus on one topic
* **Rating:** 7/10

**Plan 2: Balanced Approach**

* **Approach:** Divide the 4 weeks into four equal parts, allocating 30% of study time to each subject. Rotate topics within each subject area (e.g., algebra and geometry in math).
* **Pros:**
	+ Better balance between subjects
	+ Reduced risk of neglecting 

In [12]:
# ToT 비즈니스 전략
tot_business_prompt = """Tree of Thoughts for business strategy.
Challenge: "스타트업의 첫 제품 출시 전략 (B2B SaaS)"

Generate 3 different approaches:
1) 각 접근법의 핵심 아이디어
2) 예상 비용과 시간
3) 리스크와 기회요인
4) 성공 가능성 평가

Choose the best strategy and explain why."""

print("=== Tree of Thoughts 비즈니스 전략 ===")
tot_business_result = run_openai(tot_business_prompt, max_completion_tokens=600)
print(tot_business_result)

=== Tree of Thoughts 비즈니스 전략 ===



## 6. ReAct (Reasoning + Acting)

**핵심 아이디어**: Reason(추론) + Action(툴 호출) + Observation(결과 반영) 루프

- 장점: 실시간 데이터/DB/계산과 결합
- 설계 팁: 툴 스키마(이름/파라미터/설명) 명확히

In [13]:
# 6. ReAct - 시뮬레이션 (Ollama는 툴 호출 미지원이므로 텍스트로 흉내)
react_simulation_prompt = """Follow ReAct pattern (Reasoning + Action + Observation).
Task: "내일 서울 날씨를 확인하고 한 줄 요약해줘"

Thought: I should check the weather for Seoul tomorrow.
Action: get_weather(city='Seoul', date='tomorrow')
Observation: (simulate) 'Sunny, 27°C, light breeze'
Thought: Now I can provide a summary.
Final Answer: 내일 서울은 맑고 27도로 쾌적한 날씨가 예상됩니다.

Now do this pattern for: "오늘 달러 환율 확인하고 투자 조언 한 줄""""

print("=== ReAct 시뮬레이션 ===")
print("Ollama:")
react_ollama = run_ollama("llama3.1:8b", react_simulation_prompt)
print(react_ollama)

print("\nOpenAI:")
react_openai = run_openai(react_simulation_prompt, max_completion_tokens=300)
print(react_openai)

SyntaxError: unterminated string literal (detected at line 11) (615955541.py, line 11)

In [ ]:
# ReAct 실제 Function Calling (OpenAI)
# 실제 도구 정의
def get_weather(city: str) -> str:
    """더미 날씨 API"""
    weather_data = {
        "서울": "맑음, 25도",
        "부산": "흐림, 28도", 
        "Seoul": "Sunny, 25°C"
    }
    return weather_data.get(city, "날씨 정보 없음")

def calculate_math(expression: str) -> str:
    """안전한 수식 계산"""
    try:
        # 안전한 수식만 계산 (eval 대신 간단한 파싱)
        if re.match(r'^[0-9+\-*/().\s]+$', expression):
            result = eval(expression)  # 실제로는 더 안전한 파서 사용 권장
            return str(result)
        else:
            return "지원되지 않는 수식"
    except:
        return "계산 오류"

# Function Calling 도구 정의
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "도시의 날씨 정보를 가져옵니다",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {"type": "string", "description": "도시 이름"}
                },
                "required": ["city"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculate_math", 
            "description": "수학 계산을 수행합니다",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {"type": "string", "description": "계산할 수식"}
                },
                "required": ["expression"]
            }
        }
    }
]

print("=== ReAct with Function Calling ===")

# 첫 번째 요청
messages = [
    {"role": "user", "content": "서울 날씨 확인하고 25 * 30 계산해서 요약해줘"}
]

response = client.chat.completions.create(
    model="gpt-5-mini",
    messages=messages,
    tools=tools,
    tool_choice="auto",
)

# 도구 호출 처리
if response.choices[0].message.tool_calls:
    messages.append(response.choices[0].message)
    
    for tool_call in response.choices[0].message.tool_calls:
        function_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        
        if function_name == "get_weather":
            result = get_weather(arguments["city"])
        elif function_name == "calculate_math":
            result = calculate_math(arguments["expression"])
        else:
            result = "Unknown function"
        
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "name": function_name,
            "content": str(result)
        })
        print(f"Tool used: {function_name} -> {result}")
    
    # 최종 응답 생성
    final_response = client.chat.completions.create(
        model="gpt-5-mini",
        messages=messages,
    )
    
    print("\nFinal Answer:")
    print(final_response.choices[0].message.content)
else:
    print("No tool calls made:")
    print(response.choices[0].message.content)

## 7. Program-Aided Language Model (PAL)

**핵심 아이디어**: 모델이 코드(예: Python)를 작성 → 실행 결과를 답변에 사용

- 장점: 수치/조합/탐색 정확도↑, 재현성↑
- 설계 팁: 안전 실행 환경(샌드박스), 금지 모듈 차단

In [ ]:
# 7. PAL - 수학 함수 생성 및 실행
pal_prompt = """Write Python function gcd_list(nums: list[int]) -> int using math.gcd and functools.reduce.
Return ONLY the code, no explanation."""

print("=== PAL 코드 생성 ===")
print("Ollama:")
pal_ollama = run_ollama("llama3.1:8b", pal_prompt)
print(pal_ollama)

print("\nOpenAI:")
pal_openai = run_openai(pal_prompt, max_completion_tokens=200)
print(pal_openai)

In [ ]:
# PAL 코드 실행 (안전한 방식)
def safe_execute_code(code: str, test_input: list) -> str:
    """안전하게 코드를 실행"""
    try:
        # 위험한 키워드 체크
        dangerous_keywords = ['import os', 'import sys', 'open(', 'exec', 'eval']
        for keyword in dangerous_keywords:
            if keyword in code:
                return f"위험한 코드 감지: {keyword}"
        
        # 허용된 import만 추가
        safe_code = "import math\nimport functools\n" + code
        
        # 코드 실행
        local_env = {}
        exec(safe_code, {"math": math, "functools": functools}, local_env)
        
        # 함수 실행
        if 'gcd_list' in local_env:
            result = local_env['gcd_list'](test_input)
            return str(result)
        else:
            return "함수 gcd_list를 찾을 수 없음"
    except Exception as e:
        return f"실행 오류: {str(e)}"

# 테스트
test_numbers = [24, 60, 36]
print(f"=== PAL 코드 실행 테스트: gcd_list({test_numbers}) ===")

# OpenAI 코드 실행
openai_result = safe_execute_code(pal_openai, test_numbers)
print(f"OpenAI 코드 실행 결과: {openai_result}")

# 정답 확인 (수동 계산)
actual_gcd = functools.reduce(math.gcd, test_numbers)
print(f"실제 GCD({test_numbers}): {actual_gcd}")

In [16]:
# PAL 복잡한 계산 예시
complex_pal_prompt = """Write Python code to solve this problem:
"사다리꼴의 넓이를 구하시오. 윗변 12cm, 아랫변 20cm, 높이 8cm"

Return executable Python code that calculates and prints the result."""

print("=== PAL 복잡한 계산 ===")
complex_code = run_openai(complex_pal_prompt, max_completion_tokens=200)
print("생성된 코드:")
print(complex_code)

# 안전 실행
def safe_execute_calculation(code: str) -> str:
    try:
        # 출력 캡처를 위한 StringIO 사용
        from io import StringIO
        import sys
        
        old_stdout = sys.stdout
        sys.stdout = captured_output = StringIO()
        
        # 코드에서 위험 요소 제거 후 실행
        clean_code = code.replace('```python', '').replace('```', '')
        exec(clean_code)
        
        sys.stdout = old_stdout
        return captured_output.getvalue().strip()
    except Exception as e:
        if 'sys' in locals():
            sys.stdout = old_stdout
        return f"실행 오류: {str(e)}"

result = safe_execute_calculation(complex_code)
print(f"\n실행 결과: {result}")

# 수동 검증
area = (12 + 20) * 8 / 2
print(f"수동 계산 검증: {area} cm²")

=== PAL 복잡한 계산 ===
생성된 코드:


실행 결과: 
수동 계산 검증: 128.0 cm²


## 기법 비교 및 성능 분석

In [ ]:
# 동일한 문제에 대한 다양한 기법 비교
comparison_problem = "23 × 47 + 15 × 32를 계산하시오"

print("=== 기법별 성능 비교: 복합 계산 문제 ===")
print(f"문제: {comparison_problem}")
print(f"정답: {23 * 47 + 15 * 32}")
print()

# 1. Zero-shot
zero_shot = run_openai(f"Calculate: {comparison_problem}", max_completion_tokens=100)
print(f"Zero-shot: {zero_shot}")

# 2. Chain of Thought
cot = run_openai(f"Solve step by step: {comparison_problem}", max_completion_tokens=200)
print(f"\nCoT: {cot}")

# 3. PAL
pal_calc_prompt = f"Write Python code to calculate: {comparison_problem}"
pal_code = run_openai(pal_calc_prompt, max_completion_tokens=150)
pal_result = safe_execute_calculation(pal_code)
print(f"\nPAL 코드: {pal_code}")
print(f"PAL 결과: {pal_result}")

In [ ]:
# 추론 품질 평가 메트릭
def evaluate_reasoning_quality(response: str, expected_answer: any) -> dict:
    """추론 품질 평가"""
    metrics = {
        'has_steps': bool(re.search(r'(step|단계|first|then|next)', response, re.I)),
        'has_calculation': bool(re.search(r'[0-9]+\s*[+\-*/×÷]\s*[0-9]+', response)),
        'has_final_answer': bool(re.search(r'(답|answer|result|결과)', response, re.I)),
        'length': len(response),
        'confidence_words': len(re.findall(r'(확실|certain|clear|obvious)', response, re.I))
    }
    
    # 숫자 정확도 체크 (간단한 버전)
    numbers = re.findall(r'\b\d+\b', response)
    if numbers:
        try:
            final_number = int(numbers[-1])  # 마지막 숫자를 최종 답으로 간주
            metrics['correct_answer'] = (final_number == expected_answer)
        except:
            metrics['correct_answer'] = False
    else:
        metrics['correct_answer'] = False
    
    return metrics

# 평가 실행
expected = 23 * 47 + 15 * 32

evaluations = {
    'Zero-shot': evaluate_reasoning_quality(zero_shot, expected),
    'CoT': evaluate_reasoning_quality(cot, expected),
    'PAL': evaluate_reasoning_quality(pal_result, expected)
}

print("\n=== 추론 품질 평가 ===")
for method, metrics in evaluations.items():
    print(f"\n{method}:")
    for metric, value in metrics.items():
        print(f"  {metric}: {value}")

## 종합 실습: 하이브리드 접근법

In [17]:
# 여러 기법을 조합한 하이브리드 시스템
def hybrid_reasoning_system(problem: str, problem_type: str = "auto") -> dict:
    """문제 유형에 따라 최적 기법을 자동 선택하는 시스템"""
    
    # 문제 유형 자동 분류
    if problem_type == "auto":
        if re.search(r'\d+\s*[+\-*/×÷]\s*\d+', problem):
            problem_type = "math"
        elif re.search(r'(전략|계획|방법|approach|strategy)', problem, re.I):
            problem_type = "planning"
        elif re.search(r'(분류|classify|sentiment)', problem, re.I):
            problem_type = "classification"
        else:
            problem_type = "general"
    
    results = {"problem_type": problem_type, "methods_used": [], "results": {}}
    
    # 문제 유형별 최적 기법 적용
    if problem_type == "math":
        # 수학 문제: PAL + CoT 조합
        results["methods_used"] = ["PAL", "CoT"]
        
        # PAL 시도
        pal_prompt = f"Write Python code to solve: {problem}"
        pal_code = run_openai(pal_prompt, max_completion_tokens=200)
        pal_result = safe_execute_calculation(pal_code)
        results["results"]["PAL"] = {"code": pal_code, "result": pal_result}
        
        # CoT 백업
        cot_prompt = f"Solve step by step with detailed reasoning: {problem}"
        cot_result = run_openai(cot_prompt, max_completion_tokens=300)
        results["results"]["CoT"] = cot_result
        
    elif problem_type == "planning":
        # 기획 문제: ToT
        results["methods_used"] = ["ToT"]
        tot_prompt = f"""Generate 3 different approaches to: {problem}
For each: outline, pros/cons, feasibility rating.
Choose the best approach."""
        tot_result = run_openai(tot_prompt, max_completion_tokens=500)
        results["results"]["ToT"] = tot_result
        
    elif problem_type == "classification":
        # 분류 문제: Few-shot
        results["methods_used"] = ["Few-shot"]
        few_shot_prompt = f"""Examples of classification:
Input: "좋은 제품이에요" -> positive
Input: "별로예요" -> negative  
Input: "보통입니다" -> neutral

Now classify: {problem}"""
        few_shot_result = run_openai(few_shot_prompt, max_completion_tokens=100)
        results["results"]["Few-shot"] = few_shot_result
        
    else:
        # 일반 문제: CoT
        results["methods_used"] = ["CoT"]
        cot_prompt = f"Think step by step and solve: {problem}"
        cot_result = run_openai(cot_prompt,max_completion_tokens=300)
        results["results"]["CoT"] = cot_result
    
    return results

# 테스트 케이스들
test_cases = [
    "125 × 8 + 75 ÷ 3을 계산하시오",
    "새로운 온라인 쇼핑몰 런칭 전략을 3가지 제안해주세요", 
    "'배송이 빨라서 만족해요'를 분류하세요",
    "인공지능의 장단점을 설명해주세요"
]

print("=== 하이브리드 추론 시스템 테스트 ===")
for i, test_case in enumerate(test_cases, 1):
    print(f"\n--- 테스트 케이스 {i} ---")
    print(f"문제: {test_case}")
    
    result = hybrid_reasoning_system(test_case)
    
    print(f"분류된 문제 유형: {result['problem_type']}")
    print(f"사용된 기법: {', '.join(result['methods_used'])}")
    
    for method, output in result['results'].items():
        if isinstance(output, dict):
            print(f"\n{method} 결과: {output.get('result', 'N/A')}")
        else:
            print(f"\n{method} 결과: {output[:200]}...")  # 처음 200자만 표시

=== 하이브리드 추론 시스템 테스트 ===

--- 테스트 케이스 1 ---
문제: 125 × 8 + 75 ÷ 3을 계산하시오
분류된 문제 유형: math
사용된 기법: PAL, CoT

PAL 결과: 

CoT 결과: ...

--- 테스트 케이스 2 ---
문제: 새로운 온라인 쇼핑몰 런칭 전략을 3가지 제안해주세요
분류된 문제 유형: planning
사용된 기법: ToT

ToT 결과: ...

--- 테스트 케이스 3 ---
문제: '배송이 빨라서 만족해요'를 분류하세요
분류된 문제 유형: classification
사용된 기법: Few-shot

Few-shot 결과: ...

--- 테스트 케이스 4 ---
문제: 인공지능의 장단점을 설명해주세요
분류된 문제 유형: general
사용된 기법: CoT

CoT 결과: ...


## 정리 및 베스트 프랙티스

### 기법별 적용 가이드

| 기법 | 적용 상황 | 장점 | 단점 |
|------|-----------|------|------|
| Zero-shot | 간단한 분류, 빠른 테스트 | 빠름, 간단 | 불안정한 포맷 |
| Few-shot | 포맷 안정화 필요시 | 일관된 출력 | 컨텍스트 소모 |
| CoT | 논리적 추론, 수학 문제 | 높은 정확도 | 긴 응답 시간 |
| Least-to-Most | 복잡한 제약 문제 | 체계적 접근 | 복잡한 설계 |
| ToT | 창의적 문제, 전략 수립 | 다양한 관점 | 높은 비용 |
| ReAct | 실시간 데이터 필요 | 최신 정보 | 도구 의존성 |
| PAL | 정확한 계산 | 100% 정확 | 코드 안전성 |

### 실무 권장사항

1. **문제 유형 자동 분류** 시스템 구축
2. **하이브리드 접근법**으로 여러 기법 조합
3. **성능 모니터링**을 통한 지속적 개선
4. **안전성 검증** (특히 PAL 사용시)
5. **비용 효율성** 고려한 기법 선택